# Set up Connection

In [9]:
import json
import pandas as pd
from datetime import date
import re
import snowflake.connector
# from snowflake.sqlalchemy import URL
# from sqlalchemy import create_engine
from snowflake.connector import ProgrammingError

# Gets the credentials
with open('/Users/cyndiz/Documents/Projects/credentials/snowflake.json') as f:
    config2 = json.load(f)     

# Connect to DB
con = snowflake.connector.connect(
    user=config2['username'],
    password=config2['password'],
    account=config2['account'],
    authenticator='externalbrowser',
    warehouse='WH_MARKETING_SCIENCE_XXL',
    database='MARKETING_SCIENCE',
    schema='LOCAL'
    )

cur = con.cursor()


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


# Read Files

In [21]:
import re
import numpy as np
from snowflake.connector.pandas_tools import write_pandas

foldername = '/Users/cyndiz/Documents/Projects/PI001-MetaMMM/ElsyMMM/Farmers/2022-06-30 11.51 init/'

filenames = [
    foldername+'model_stats.csv',
    foldername+'Weekly_long.csv',
    foldername+'ScenarioPlanning.csv',
    foldername+'pareto_alldecomp_matrix.csv'
]
filenames

['/Users/cyndiz/Documents/Projects/PI001-MetaMMM/ElsyMMM/Farmers/2022-06-30 11.51 init/model_stats.csv',
 '/Users/cyndiz/Documents/Projects/PI001-MetaMMM/ElsyMMM/Farmers/2022-06-30 11.51 init/Weekly_long.csv',
 '/Users/cyndiz/Documents/Projects/PI001-MetaMMM/ElsyMMM/Farmers/2022-06-30 11.51 init/ScenarioPlanning.csv',
 '/Users/cyndiz/Documents/Projects/PI001-MetaMMM/ElsyMMM/Farmers/2022-06-30 11.51 init/pareto_alldecomp_matrix.csv']

In [22]:
database='MARKETING_SCIENCE'
schema='CZHONG'
client='DEMO'
df_sp = pd.read_csv(foldername+'ScenarioPlanning.csv')
sol_id = list(df_sp['solID'].unique())
start_ds = df_sp['ds'].min()
end_ds = df_sp['ds'].max()
date_range = 'From %s to %s with covid only'%(start_ds,end_ds)

cur.execute('''create or replace file format myformat type='CSV' skip_header=1;''')
cur.execute('''create or replace stage stage_mmm file_format=myformat copy_options = (on_error='skip_file');''')
for filename in filenames:
    print(filename)
    # qry_put= '''PUT file://{filename} @stage_mmm;'''.format(filename=filename)
    filename_model = pd.read_csv(filename)
    filename_model.replace([np.inf, -np.inf],0, inplace=True)
    filename_model['client']=client
    filename_model.columns = map(str.upper, filename_model.columns)
    if re.search('model_stats.csv',filename):
        # tablename = 'MARKETING_SCIENCE.CZHONG.MMM_MODEL_STATS'
        tablename='MMM_MODEL_STATS'
        filename_model=filename_model.rename(columns={'DECOMP.RSSD':'DECOMP_RSSD'})
        # print(filename_model.loc[filename_model['SOLID']=='5_238_6'])
        filename_model['DATE_RANGE']=date_range
        names = filename_model.columns
        formatted_names = list(map(lambda x: ('{x} varchar' if (x in ('SOLID','RN','CLIENT','DATE_RANGE')) else '{x} float').format(x=x), names))
    elif re.search('Weekly_long.csv', filename):
        # tablename = 'MARKETING_SCIENCE.CZHONG.MMM_RAW'
        tablename = 'MMM_RAW'
        filename_model=filename_model.rename(columns={'BM.SALES_CALLS.PRD.54':'SALES_CALLS'})
        # filename_model['DATE']=pd.to_datetime(filename_model['DATE'])
        names = filename_model.columns
        formatted_names = list(map(lambda x: ('{x} varchar' if (x in ('METRICS','CLIENT')) else ('{x} date' if x=='DATE' else '{x} float')).format(x=x), names))
    elif re.search('ScenarioPlanning.csv',filename):
        # tablename = 'MARKETING_SCIENCE.CZHONG.MMM_SCENARIO_PLANNING'
        tablename = 'MMM_SCENARIO_PLANNING'
        # filename_model['DS']=pd.to_datetime(filename_model['DS'])
        filename_model['DATE_RANGE']=date_range
        names = filename_model.columns
        formatted_names = list(map(lambda x: ('{x} varchar' if (x in ('CHANNEL','SOLID','CLIENT','TYPE','DATE_RANGE')) else ('{x} date' if x=='DS' else ('{x} boolean' if x=='FLAG_MEAN' else '{x} float'))).format(x=x), names))
    elif re.search('pareto_alldecomp_matrix.csv',filename):
        tablename = 'MMM_RESPONSE'
        filename_model = filename_model.loc[filename_model['SOLID'].isin(sol_id)]
        filename_model = filename_model[['DS','DEP_VAR','DEPVARHAT','CLIENT','SOLID']]
        filename_model['DATE_RANGE']=date_range
        names = filename_model.columns
        formatted_names = list(map(lambda x: ('{x} varchar' if (x in ('CHANNEL','SOLID','CLIENT','TYPE','DATE_RANGE')) else ('{x} date' if x=='DS' else ('{x} boolean' if x=='FLAG_MEAN' else '{x} float'))).format(x=x), names))
    col_n = ',\n'.join(formatted_names)
    print(col_n)
    qry_create = '''CREATE TABLE IF NOT EXISTS {database}.{schema}.{tablename} ({col_n}) CLUSTER BY (CLIENT); '''.format(col_n=col_n,tablename=tablename,database=database,schema=schema)
    # qry_create = '''CREATE OR REPLACE TABLE {database}.{schema}.{tablename} ({col_n}) CLUSTER BY (CLIENT); '''.format(col_n=col_n,tablename=tablename,database=database,schema=schema)
    if tablename in ('MMM_RESPONSE','MMM_SCENARIO_PLANNING','MMM_MODEL_STATS'):
        qry_delete = '''DELETE FROM {database}.{schema}.{tablename} WHERE client='{client}' and date_range='{date_range}';'''.format(tablename=tablename,database=database,schema=schema,date_range=date_range,client=client)
    else:
        qry_delete = '''DELETE FROM {database}.{schema}.{tablename} WHERE client='{client}';'''.format(tablename=tablename,database=database,schema=schema,client=client)
    # qry_copy = '''COPY INTO {tablename} from @stage_mmm;'''.format(tablename=tablename)
    # cur.execute(qry_put)
    cur.execute(qry_create)
    # print('Table created!')
    cur.execute(qry_delete)
    # cur.execute(qry_copy)
    success, nchunks, nrows, _ = write_pandas(con, filename_model, '{tablename}'.format(tablename=tablename),database='MARKETING_SCIENCE',schema='CZHONG')
    cur.execute('''select * from {tablename} limit 10;'''.format(tablename=database+'.'+schema+'.'+tablename))
    rows=cur.fetch_pandas_all()
    print(rows)
print('Upload Successfully!')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/cyndiz/Documents/Projects/PI001-MetaMMM/ElsyMMM/Farmers/2022-06-30 11.51 init/ScenarioPlanning.csv'

In [18]:
cur.execute('select * from marketing_science.czhong.mmm_response;')
rows=cur.fetch_pandas_all()
display(rows)

,DS,DEP_VAR,DEPVARHAT,CLIENT,SOLID,DATE_RANGE
0,2020-01-26,86019.0,84142.960924,Farmer,1_222_8,From 2018-12-30 to 2022-05-29
1,2019-03-10,93981.0,104633.156599,Farmer,1_222_8,From 2018-12-30 to 2022-05-29
2,2019-04-07,85773.0,95542.161305,Farmer,1_222_8,From 2018-12-30 to 2022-05-29
3,2019-06-02,90005.0,82330.035845,Farmer,1_222_8,From 2018-12-30 to 2022-05-29
4,2019-09-01,79936.0,75183.817938,Farmer,1_222_8,From 2018-12-30 to 2022-05-29
...,...,...,...,...,...,...
38838,2022-02-21,2193.0,2809.758698,Farmer,5_92_7,From 2018-12-31 to 2022-05-30
38839,2022-04-25,2905.0,2835.133773,Farmer,5_92_7,From 2018-12-31 to 2022-05-30
38840,2022-05-16,3209.0,2932.251806,Farmer,5_92_7,From 2018-12-31 to 2022-05-30
38841,2021-09-27,3038.0,2509.181594,Farmer,5_92_7,From 2018-12-31 to 2022-05-30


In [12]:
date_range

'****From 2018-12-31 to 2022-05-30'

In [29]:
len(filename_model)

23628

In [34]:
sol_id = list(df_sp['solID'].unique())
a=filename_model.loc[filename_model['SOLID'].isin(sol_id)]
len(a)
a

,DS,DEP_VAR,DEPVARHAT,CLIENT,SOLID,DATE_RANGE
1611,2018-12-31,3371,3941.507189,Farmer,2_171_2,From 2018-12-31 to 2022-05-30
1612,2019-01-07,4281,4403.513312,Farmer,2_171_2,From 2018-12-31 to 2022-05-30
1613,2019-01-14,4382,4469.551877,Farmer,2_171_2,From 2018-12-31 to 2022-05-30
1614,2019-01-21,4280,4065.005199,Farmer,2_171_2,From 2018-12-31 to 2022-05-30
1615,2019-01-28,4604,4376.337891,Farmer,2_171_2,From 2018-12-31 to 2022-05-30
...,...,...,...,...,...,...
18969,2022-05-02,3113,2827.571824,Farmer,3_217_2,From 2018-12-31 to 2022-05-30
18970,2022-05-09,3049,2866.271853,Farmer,3_217_2,From 2018-12-31 to 2022-05-30
18971,2022-05-16,3209,2924.412086,Farmer,3_217_2,From 2018-12-31 to 2022-05-30
18972,2022-05-23,3277,2895.634100,Farmer,3_217_2,From 2018-12-31 to 2022-05-30
